# Verily Workbench Notebook Cost Estimator

This tool helps you estimate the approximate GCP cost of running a specified JupyterLab notebook on a standard Verily Workbench JupyterLab app. 

Refer to the [Verily Workbench Cloud Apps documentation](https://support.workbench.verily.com/docs/guides/cloud_apps/apps_intro/) for details on default app configurations and pricing.

In [ ]:
# Import Required Libraries
import os
import math
import pandas as pd
from pathlib import Path
from IPython.display import display, Markdown

## User Input: Notebook and Data Specifications

Please provide the following information to estimate your cost:

In [ ]:
# User Input: Notebook and Data Specifications
from ipywidgets import widgets

notebook_file = widgets.Text(
    value='',
    placeholder='Enter notebook filename (e.g., analysis.ipynb)',
    description='Notebook:',
    disabled=False
)
runtime_hours = widgets.FloatText(
    value=1.0,
    description='Runtime (hrs):',
    disabled=False
)

# Workspace resource inputs
resource_type = widgets.Dropdown(
    options=[
        ('BigQuery Dataset', 'bq_dataset'),
        ('BigQuery Table', 'bq_table'), 
        ('Cloud Storage Bucket', 'gcs_bucket'),
        ('Cloud Storage Object/File', 'gcs_object'),
        ('Mixed Resources', 'mixed')
    ],
    value='gcs_bucket',
    description='Resource Type:',
    disabled=False
)

data_size_gb = widgets.FloatText(
    value=1.0,
    description='Data Size (GB):',
    disabled=False
)
output_size_gb = widgets.FloatText(
    value=0.5,
    description='Output Size (GB):',
    disabled=False
)

# BigQuery-specific inputs
bq_queries = widgets.IntText(
    value=1,
    description='# BQ Queries:',
    disabled=False
)
bq_data_processed_gb = widgets.FloatText(
    value=1.0,
    description='BQ Data Processed (GB):',
    disabled=False
)

special_resources = widgets.Text(
    value='',
    placeholder='e.g., GPU, highmem',
    description='Special Resources:',
    disabled=False
)

ui = widgets.VBox([
    notebook_file, 
    runtime_hours, 
    resource_type,
    data_size_gb, 
    output_size_gb,
    bq_queries,
    bq_data_processed_gb,
    special_resources
])
display(ui)

## Estimate Compute Resource Usage

This section estimates the compute resources required based on your input and the default JupyterLab app specs from Verily Workbench documentation.

In [ ]:
# Estimate Compute Resource Usage
# Default JupyterLab app specs (as of Nov 2025, see Verily docs):
# n1-standard-4 (4 vCPU, 15 GB RAM), $0.158/hr (us-central1)
default_machine_type = 'n1-standard-4'
default_vcpu = 4
default_ram_gb = 15
compute_price_per_hour = 0.158  # USD/hr (update if pricing changes)

print(f"Default machine: {default_machine_type} ({default_vcpu} vCPU, {default_ram_gb} GB RAM)")
print(f"Compute price: ${compute_price_per_hour}/hr (us-central1, Nov 2025)")

## Estimate Storage Usage Based on Workspace Resources

Estimate storage usage based on the data resources in your Verily Workbench workspace. Workbench supports different types of data resources:

- **BigQuery Datasets/Tables**: Query-based pricing
- **Cloud Storage Buckets/Objects**: Storage + data transfer pricing  
- **Referenced Resources**: Point to external data (no workspace storage cost)
- **Controlled Resources**: Managed within your workspace

For details, see the [Workbench data resources documentation](https://support.workbench.verily.com/docs/guides/research_data/resource_intro/).

In [ ]:
# Estimate Storage Usage Based on Workspace Resources
# GCP Pricing (us-central1, Nov 2025 - update as needed)
storage_price_per_gb_month = 0.04  # Cloud Storage Standard
bq_query_price_per_tb = 6.25  # BigQuery on-demand pricing per TB processed
bq_storage_price_per_gb_month = 0.02  # BigQuery storage per GB/month

# Convert monthly to hourly rates
storage_price_per_gb_hour = storage_price_per_gb_month / (30 * 24)
bq_storage_price_per_gb_hour = bq_storage_price_per_gb_month / (30 * 24)

def estimate_storage_costs(resource_type, data_gb, output_gb, queries, processed_gb, runtime_hrs):
    storage_cost = 0
    query_cost = 0
    
    if resource_type in ['gcs_bucket', 'gcs_object', 'mixed']:
        # Cloud Storage costs for data + output
        total_storage_gb = data_gb + output_gb
        storage_cost = storage_price_per_gb_hour * total_storage_gb * runtime_hrs
        
    elif resource_type in ['bq_dataset', 'bq_table']:
        # BigQuery storage (for controlled datasets) + query costs
        storage_cost = bq_storage_price_per_gb_hour * data_gb * runtime_hrs
        query_cost = (processed_gb / 1000) * bq_query_price_per_tb * queries  # Convert GB to TB
        
    elif resource_type == 'mixed':
        # Combination of storage and BigQuery
        gcs_storage = storage_price_per_gb_hour * data_gb * runtime_hrs
        bq_queries = (processed_gb / 1000) * bq_query_price_per_tb * queries
        storage_cost = gcs_storage + bq_queries
    
    return storage_cost, query_cost

storage_cost, query_cost = estimate_storage_costs(
    resource_type.value,
    data_size_gb.value,
    output_size_gb.value, 
    bq_queries.value,
    bq_data_processed_gb.value,
    runtime_hours.value
)

total_data_cost = storage_cost + query_cost

print(f"Resource type: {resource_type.value}")
print(f"Estimated storage cost: ${storage_cost:.4f}")
if query_cost > 0:
    print(f"Estimated BigQuery query cost: ${query_cost:.4f}")
print(f"Total data-related cost: ${total_data_cost:.4f}")

## Calculate Approximate Cost

This section calculates the estimated cost for compute and storage resources based on your inputs and current pricing.

In [ ]:
# Calculate Approximate Cost
def estimate_total_cost(runtime_hours, storage_cost, query_cost):
    compute_cost = compute_price_per_hour * runtime_hours
    total_data_cost = storage_cost + query_cost
    total_cost = compute_cost + total_data_cost
    return compute_cost, total_data_cost, total_cost

compute_cost, total_data_cost, total_cost = estimate_total_cost(
    runtime_hours.value, 
    storage_cost, 
    query_cost
)

print(f"Estimated compute cost: ${compute_cost:.2f}")
print(f"Estimated data cost (storage + queries): ${total_data_cost:.4f}")
print(f"Total estimated cost: ${total_cost:.2f}")

# Cost breakdown by component
if resource_type.value in ['bq_dataset', 'bq_table'] and query_cost > 0:
    print(f"\nData cost breakdown:")
    print(f"  - Storage: ${storage_cost:.4f}")
    print(f"  - BigQuery queries: ${query_cost:.4f}")

## Display Cost Breakdown

Below is a detailed breakdown of your estimated costs for running the specified notebook on Verily Workbench.

In [ ]:
# Display Cost Breakdown
import pandas as pd

# Create cost breakdown table
cost_components = ['Compute', 'Data (Storage + Queries)', 'Total']
cost_values = [f"${compute_cost:.2f}", f"${total_data_cost:.4f}", f"${total_cost:.2f}"]

cost_breakdown = pd.DataFrame({
    'Component': cost_components,
    'Estimated Cost (USD)': cost_values
})

display(cost_breakdown)

# Detailed explanation
resource_explanation = {
    'gcs_bucket': 'Cloud Storage bucket',
    'gcs_object': 'Cloud Storage object/file', 
    'bq_dataset': 'BigQuery dataset',
    'bq_table': 'BigQuery table',
    'mixed': 'Mixed resources (Cloud Storage + BigQuery)'
}

explanation = f"""
**Cost Estimation Details:**

**Compute:**
- Default JupyterLab app: {default_machine_type} ({default_vcpu} vCPU, {default_ram_gb} GB RAM)
- Compute price: ${compute_price_per_hour}/hr (us-central1, Nov 2025)
- Runtime: {runtime_hours.value} hours

**Data Resources:**
- Resource type: {resource_explanation.get(resource_type.value, resource_type.value)}
- Storage price: ${storage_price_per_gb_month}/GB/month (Cloud Storage Standard)"""

if resource_type.value in ['bq_dataset', 'bq_table']:
    explanation += f"""
- BigQuery storage: ${bq_storage_price_per_gb_month}/GB/month
- BigQuery queries: ${bq_query_price_per_tb}/TB processed
- Estimated queries: {bq_queries.value}
- Data processed per query: {bq_data_processed_gb.value} GB"""

explanation += f"""

**Notes:**
- Costs are estimated for us-central1 region
- Referenced resources (external data) may have no workspace storage cost
- Controlled resources are managed within your workspace
- Actual costs may vary based on region, usage patterns, and discounts
- See [Workbench data resources](https://support.workbench.verily.com/docs/guides/research_data/resource_intro/) for more details
"""

display(Markdown(explanation))

## Discover Your Workspace Resources with wb CLI

The Verily Workbench CLI is available in this environment. We'll automatically discover and analyze the data resources in your workspace to provide accurate size estimates and resource types for cost calculation.

In [ ]:
# Discover Your Workspace Resources using Workbench CLI
import subprocess
import json
import pandas as pd
from IPython.display import display, HTML

def run_wb_command(command):
    """Run a wb CLI command and return the result"""
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        return result.stdout.strip()
    else:
        raise Exception(f"Command failed: {command}\nError: {result.stderr}")

def list_workspace_resources():
    """List all resources in the current workspace"""
    print("🔍 Discovering workspace resources...")
    
    # List all resources in JSON format for easier parsing
    resources_json = run_wb_command("wb resource list --format=JSON")
    
    resources = json.loads(resources_json)
    if not resources:
        print("No resources found in this workspace.")
        return pd.DataFrame()
    
    # Create a summary table
    resource_data = []
    for resource in resources:
        resource_data.append({
            'Name': resource.get('name', 'Unknown'),
            'Type': resource.get('resourceType', 'Unknown'),
            'Stewardship': resource.get('stewardshipType', 'Unknown'),
            'Description': resource.get('description', '')[:50] + '...' if len(resource.get('description', '')) > 50 else resource.get('description', ''),
            'Cloud Resource': resource.get('cloudName', 'Unknown')
        })
    
    df = pd.DataFrame(resource_data)
    print(f"\n✅ Found {len(resources)} resources in your workspace:")
    display(df)
    return df

def get_resource_details(resource_name):
    """Get detailed information about a specific resource"""
    print(f"📋 Getting details for resource: {resource_name}")
    
    details = run_wb_command(f"wb resource describe --name={resource_name} --format=JSON")
    resource_info = json.loads(details)
    
    print(f"\n**Resource:** {resource_info.get('name', 'Unknown')}")
    print(f"**Type:** {resource_info.get('resourceType', 'Unknown')}")
    print(f"**Stewardship:** {resource_info.get('stewardshipType', 'Unknown')}")
    print(f"**Cloud Resource:** {resource_info.get('cloudName', 'Unknown')}")
    
    # For GCS buckets, we can try to get size info
    if resource_info.get('resourceType') == 'GCS_BUCKET':
        bucket_name = resource_info.get('cloudName', '')
        if bucket_name:
            print(f"**Bucket:** {bucket_name}")
            print(f"💡 Use get_gcs_bucket_size('{bucket_name}') to get actual size")
    
    # For BigQuery datasets, show how to get table info
    elif resource_info.get('resourceType') == 'BQ_DATASET':
        dataset_name = resource_info.get('cloudName', '')
        if dataset_name:
            print(f"**Dataset:** {dataset_name}")
            print(f"💡 Use get_bigquery_dataset_size('{dataset_name}') to get actual size")
    
    return resource_info

# Automatically run the resource discovery
resources_df = list_workspace_resources()

In [ ]:
# Get detailed information for specific resources
print("📋 Available resources in your workspace:")
if not resources_df.empty:
    for i, name in enumerate(resources_df['Name'], 1):
        resource_type = resources_df.iloc[i-1]['Type']
        stewardship = resources_df.iloc[i-1]['Stewardship']
        print(f"   {i}. {name} ({resource_type}, {stewardship})")
    
    print(f"\n💡 To get detailed information about a resource, use:")
    print(f"   resource_details = get_resource_details('resource-name')")
    print(f"\n💡 Example: get_resource_details('{resources_df.iloc[0]['Name']}')")
else:
    print("   No resources found in this workspace.")
    print("   Make sure you're running this in a Verily Workbench JupyterLab environment with resources configured.")

In [ ]:
# Get Cloud Storage bucket sizes and BigQuery dataset information
def get_gcs_bucket_size(bucket_name):
    """Get the size of a GCS bucket using gsutil via wb CLI"""
    print(f"📏 Getting size for bucket: {bucket_name}")
    
    # Get bucket size in bytes
    size_output = run_wb_command(f"wb gsutil du -s gs://{bucket_name}")
    
    # Parse the size (gsutil du returns: size_bytes gs://bucket_name)
    if size_output:
        size_bytes = int(size_output.split()[0])
        size_gb = size_bytes / (1024**3)  # Convert to GB
        
        print(f"Bucket size: {size_gb:.2f} GB ({size_bytes:,} bytes)")
        return size_gb
    else:
        print(f"Could not determine bucket size")
        return 0

def get_bigquery_dataset_size(project_dataset):
    """Get BigQuery dataset size and table information"""
    print(f"📊 Getting BigQuery dataset info: {project_dataset}")
    
    # Split project.dataset if needed
    if '.' not in project_dataset:
        print("⚠️ Dataset name should be in format 'project.dataset' or provide the full dataset reference")
        return 0
    
    project, dataset = project_dataset.split('.', 1)
    
    # Query to get table sizes
    sql_query = f"""
    SELECT 
        table_name,
        ROUND(size_bytes/1024/1024/1024, 2) as size_gb,
        row_count,
        creation_time
    FROM `{project}.{dataset}.INFORMATION_SCHEMA.TABLES`
    WHERE table_type = 'BASE TABLE'
    ORDER BY size_bytes DESC
    """
    
    print("Running BigQuery size analysis...")
    query_result = run_wb_command(f'wb bq query --sql="{sql_query}" --format=JSON')
    
    tables = json.loads(query_result)
    
    if tables:
        print(f"\n📋 Tables in {project_dataset}:")
        table_data = []
        total_size_gb = 0
        
        for table in tables:
            size_gb = float(table.get('size_gb', 0))
            total_size_gb += size_gb
            table_data.append({
                'Table': table.get('table_name', ''),
                'Size (GB)': size_gb,
                'Rows': table.get('row_count', 0),
                'Created': table.get('creation_time', '')[:10]  # Just date
            })
        
        df = pd.DataFrame(table_data)
        display(df)
        print(f"\n**Total dataset size: {total_size_gb:.2f} GB**")
        return total_size_gb
    else:
        print("No tables found in dataset")
        return 0

# Auto-detect and analyze resources with actual sizes
print("🔍 Analyzing discovered resources for accurate size information...")

for _, resource in resources_df.iterrows():
    resource_name = resource['Name']
    resource_type = resource['Type']
    cloud_resource = resource['Cloud Resource']
    
    print(f"\n🔹 **{resource_name}** ({resource_type})")
    
    try:
        if resource_type == 'GCS_BUCKET' and cloud_resource != 'Unknown':
            actual_size = get_gcs_bucket_size(cloud_resource)
            print(f"   ✅ Actual size determined: {actual_size:.2f} GB")
            
        elif resource_type == 'BQ_DATASET' and cloud_resource != 'Unknown':
            actual_size = get_bigquery_dataset_size(cloud_resource)
            print(f"   ✅ Actual size determined: {actual_size:.2f} GB")
            
        else:
            print(f"   ℹ️ Size analysis not available for {resource_type}")
            
    except Exception as e:
        print(f"   ⚠️ Could not determine size: {str(e)}")

print(f"\n💡 You can also call these functions manually:")
print(f"   get_gcs_bucket_size('bucket-name')")
print(f"   get_bigquery_dataset_size('project.dataset')")

In [ ]:
# Automated Cost Estimation with Discovered Resources
def estimate_costs_from_resources():
    """Estimate costs based on discovered workspace resources"""
    print("🎯 Automated Cost Estimation Based on Your Workspace Resources")
    print("=" * 60)
    
    if resources_df.empty:
        print("No resources discovered in this workspace.")
        return 0
    
    total_estimated_cost = 0
    total_storage_gb = 0
    total_query_costs = 0
    
    print("\n📊 Analyzing each resource:")
    
    for _, resource in resources_df.iterrows():
        resource_name = resource['Name']
        resource_type = resource['Type']
        stewardship = resource['Stewardship']
        cloud_resource = resource['Cloud Resource']
        
        print(f"\n🔹 **{resource_name}** ({resource_type}, {stewardship})")
        
        estimated_size = 0
        estimated_queries = 0
        
        # Try to get actual sizes where possible
        try:
            if resource_type == 'GCS_BUCKET' and cloud_resource != 'Unknown':
                estimated_size = get_gcs_bucket_size(cloud_resource)
                print(f"   ? Actual size: {estimated_size:.2f} GB")
                
            elif resource_type == 'BQ_DATASET' and cloud_resource != 'Unknown':
                estimated_size = get_bigquery_dataset_size(cloud_resource)
                estimated_queries = max(1, int(estimated_size / 10))  # Assume 1 query per 10GB
                print(f"   📏 Actual size: {estimated_size:.2f} GB")
                print(f"   ? Estimated queries: {estimated_queries}")
                
            elif resource_type == 'GCS_OBJECT':
                estimated_size = 0.1  # Assume 100MB for individual objects
                print(f"   ? Estimated size: {estimated_size} GB")
                
            elif resource_type == 'BQ_TABLE':
                estimated_size = 0.5  # Assume 500MB for individual tables
                estimated_queries = 1
                print(f"   📏 Estimated size: {estimated_size} GB")
                print(f"   🔍 Estimated queries: {estimated_queries}")
                
            else:
                print(f"   ℹ️ Resource type {resource_type} not included in cost calculation")
                continue
                
        except Exception as e:
            print(f"   ⚠️ Could not get actual size, using estimates: {str(e)}")
            # Fall back to estimates
            if resource_type == 'GCS_BUCKET':
                estimated_size = 5.0
            elif resource_type == 'BQ_DATASET':
                estimated_size = 2.0
                estimated_queries = 5
            elif resource_type in ['GCS_OBJECT', 'BQ_TABLE']:
                estimated_size = 1.0
                estimated_queries = 1 if resource_type == 'BQ_TABLE' else 0
            
            print(f"   📏 Estimated size: {estimated_size} GB")
            if estimated_queries > 0:
                print(f"   🔍 Estimated queries: {estimated_queries}")
        
        # Calculate costs for this resource
        if resource_type in ['GCS_BUCKET', 'GCS_OBJECT']:
            storage_cost = storage_price_per_gb_hour * estimated_size * runtime_hours.value
            total_estimated_cost += storage_cost
            total_storage_gb += estimated_size
            print(f"   💰 Storage cost: ${storage_cost:.4f}")
            
        elif resource_type in ['BQ_DATASET', 'BQ_TABLE']:
            bq_storage_cost = bq_storage_price_per_gb_hour * estimated_size * runtime_hours.value
            query_cost = (estimated_size / 1000) * bq_query_price_per_tb * estimated_queries
            total_estimated_cost += bq_storage_cost + query_cost
            total_storage_gb += estimated_size
            total_query_costs += query_cost
            print(f"   💰 Storage cost: ${bq_storage_cost:.4f}")
            print(f"   💰 Query cost: ${query_cost:.4f}")
    
    # Add compute costs
    compute_cost = compute_price_per_hour * runtime_hours.value
    total_estimated_cost += compute_cost
    
    print(f"\n" + "=" * 60)
    print(f"📋 **TOTAL COST ESTIMATE FROM WORKSPACE RESOURCES**")
    print(f"   💻 Compute cost ({runtime_hours.value}h): ${compute_cost:.2f}")
    print(f"   💾 Storage cost ({total_storage_gb:.1f}GB): ${total_estimated_cost - compute_cost - total_query_costs:.4f}")
    if total_query_costs > 0:
        print(f"   🔍 Query costs: ${total_query_costs:.4f}")
    print(f"   🎯 **TOTAL: ${total_estimated_cost:.2f}**")
    
    return total_estimated_cost

# Run automated estimation with discovered resources
print("\n🚀 Running automated cost estimation...")
automated_cost = estimate_costs_from_resources()